### Generate knowledge graph and support queries using Neo4j
##### Dependency - install py2neo
##### Prerequisite - Install and have a graph instance running in Neo4j

#### Install Py2neo

In [12]:
#from py2neo import Graph
from py2neo import Database, Graph, Node, Relationship
import pickle
import pandas as pd

### Load Data

In [13]:
svos = pd.read_csv("svos.csv", names = ['sub', 'rel', 'obj'])
#print(svos)
pickle_in = open("entity_dict.pickle","rb")
entities = pickle.load(pickle_in)
#print(entities)

### Graph construction

In [14]:
# Connect to Neo4j using corresponding <port:7687> and <password>
graph = Graph("neo4j@bolt://localhost:7687",password="1432")
#graph.delete_all()

# Parse the entities and build the knowledge graph in Neo4j Database
for index, row in svos.iterrows():
    sub, rel, obj = row
    
    sub_node = graph.nodes.match(entities.get(sub, "Object"), name=sub).first()
    obj_node = graph.nodes.match("Object", name=obj).first()
    if not sub_node:
        sub_node = Node(entities.get(sub, "Object"),name = sub)
    if not obj_node:
        obj_node = Node(entities.get(obj, "Object"),name = obj)
    relation = Relationship.type(rel)    
    graph.merge(relation(sub_node, obj_node), entities.get(sub, "Object"), "name" )